In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [9]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [10]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 20595 unique tokens.


In [11]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [12]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [14]:
# prepare embedding matrix  (matrix of embeddings of the words in our dataset)
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Dense, Dropout, LSTM, Bidirectional
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [18]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')             ## input layer
embedded_sequences = embedding_layer(sequence_input)                            ## embeddings layer
X = LSTM(128, return_sequences=True)(embedded_sequences)
X = Dropout(0.5)(X)
X = LSTM(128, return_sequences=False)(X)
X = Dropout(0.5)(X)
X = Dense(30, activation='relu')(X)                                      
X = Dropout(0.5)(X)                            
preds = Dense(2, activation='softmax')(X)                                       ## softmax layer

model = Model(sequence_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras.utils import plot_model
plot_model(model, to_file='/content/drive/My Drive/Sarcasm Detection/Plots/LSTM.png')

In [20]:
model.fit(x_train, y_train, batch_size=32, epochs=15)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
5002/5002 [==============================] - 19s 4ms/step - loss: 0.3934 - acc: 0.8159
Epoch 2/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.2453 - acc: 0.8990
Epoch 3/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.2032 - acc: 0.9230
Epoch 4/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.1737 - acc: 0.9314
Epoch 5/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.1443 - acc: 0.9478
Epoch 6/15
5002/5002 [==============================] - 18s 4ms/step - loss: 0.1215 - acc: 0.9532
Epoch 7/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.1084 - acc: 0.9616
Epoch 8/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0724 - acc: 0.9694
Epoch 9/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0715 - acc: 0.9728
Epoch 10/15
5002/5002 [==============================] - 17s 3ms/step 

In [21]:
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

1667/1667 [==============================] - 3s 2ms/step
test loss is: 0.5282816817367095
test accuracy is:  0.9178164367126574


In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
       
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
       
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


model.compile(loss='categorical_crossentropy',
          optimizer= "rmsprop",
          metrics=[f1])

In [23]:
model.fit(x_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
5002/5002 [==============================] - 19s 4ms/step - loss: 0.0287 - f1: 0.9922
Epoch 2/15
5002/5002 [==============================] - 18s 4ms/step - loss: 0.0202 - f1: 0.9942
Epoch 3/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0216 - f1: 0.9940
Epoch 4/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0214 - f1: 0.9944
Epoch 5/15
5002/5002 [==============================] - 18s 4ms/step - loss: 0.0121 - f1: 0.9976
Epoch 6/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0202 - f1: 0.9956
Epoch 7/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0153 - f1: 0.9960
Epoch 8/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0109 - f1: 0.9960
Epoch 9/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0105 - f1: 0.9958
Epoch 10/15
5002/5002 [==============================] - 18s 4ms/step - loss: 0.0133 - f1: 0.9978
Epoch 11/15
5002/5002 [======

In [24]:
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

1667/1667 [==============================] - 2s 1ms/step
test loss is: 0.7533473481627763
test f1 score is:  0.9226154219839626


In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 34)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 34, 300)           6178800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 34, 128)           219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 34, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                3870      
__________